In [1]:
PYTORCH_MPS_HIGH_WATERMARK_RATIO=0

In [2]:
from rouge_score import rouge_scorer

%pip install rouge-score
%pip install --upgrade openpyxl
%pip install pandas openpyxl
%pip install transformers

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
import torch
import random
from transformers import AutoModelForCausalLM, AutoTokenizer
import pandas as pd
import os
import gc
import torch
os.environ["PYTORCH_MPS_HIGH_WATERMARK_RATIO"] = "0.0"

/Users/darshan/Documents/GitHub/ipl-sentiment-betting/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
model_name = "Qwen/Qwen2.5-3B-Instruct"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

Loading checkpoint shards: 100%|██████████| 2/2 [00:04<00:00,  2.48s/it]


In [5]:
df = pd.read_json('/Users/darshan/Documents/GitHub/ipl-sentiment-betting/model/chunks/2.json')
chunks = df["chunks"].tolist()

In [6]:
def generate_model_response(system_prompt, user_prompt):
    # Truncate long inputs to prevent OOM
    max_input_length = 512
    messages = [
        {"role": "system", "content": system_prompt[:max_input_length]},
        {"role": "user", "content": user_prompt[:max_input_length]}
    ]
    
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    
    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)
    
    with torch.no_grad():  # Reduce memory usage during inference
        generated_ids = model.generate(
            **model_inputs,
            max_new_tokens=128,  # Reduced from 512
            pad_token_id=tokenizer.eos_token_id,
            do_sample=False  # Use greedy decoding to save memory
        )
    
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]
    
    # Clear GPU memory
    del model_inputs
    gc.collect()
    torch.mps.empty_cache() if hasattr(torch.mps, 'empty_cache') else None
    
    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
    return response[0].strip().lower()

In [7]:
def analyze_sentiment(comment_text):
    system_prompt = (
        "You are a professional T20 cricket bettor."
        "You will be analyzing people's sentiment from Indian Premier League (IPL) 2024 cricket match comments from a Reddit thread"
        "The purpose of this analysis is to understand the sentiment for placing bets on the ongoing match."
        "Classify each comment as positive, negative, or neutral in the context of IPL and the ongoing match."
    )
    user_prompt = f"Comment: {comment_text}\nWhat is the sentiment? Answer in only word (positive, negative, or neutral)"
    # print(f"Comment: {comment_text}")
    sentiment = generate_model_response(system_prompt, user_prompt)
    # print(f"Sentiment: {sentiment}")
    return sentiment

In [8]:
def analyze_betting_opportunity(chunk, ball_by_ball_data):
    system_prompt = (
        "Punjab Kings Playing XI: S Dhawan, JM Bairstow, LS Livingstone, JM Sharma†, SM Curran, HV Patel, "
        "Harpreet Brar, Shashank Singh, K Rabada, RD Chahar, Arshdeep Singh\n"
        "Delhi Capitals Playing XI: MR Marsh, DA Warner, SD Hope, RR Pant*†, T Stubbs, AR Patel, "
        "Sumit Kumar, Kuldeep Yadav, RK Bhui, KK Ahmed, I Sharma\n"
        "You are a professional cricket bettor in the T20 format. Your livelihood depends on the money earned by winning bets. "
        "Analyze the following reddit comments, odds, and ball-by-ball data in this 5 minute interval to determine if you should bet or not bet. "
        "If you decide to bet, mention the team name (Delhi Capitals or Punjab Kings). Explain why that team will win this match. "
        "Limit your response to 100 words."
    )  # Note: string concatenation happens automatically for adjacent string literals
    
    user_prompt = f"""Comments: {chunk.get("comments", [])}
    Odds: {chunk.get("odds", [])}
    Ball-by-ball data: {ball_by_ball_data}
    Is there a betting opportunity? If yes, which team should I bet on?"""
    print("Analyzing betting opportunity with ball-by-ball data.")
    betting_opportunity = generate_model_response(system_prompt, user_prompt)
    print(f"Betting opportunity result: {betting_opportunity}")
    return betting_opportunity

In [9]:
def process_chunks(chunks):
    sentiments = []
    betting_opportunities = []
    batch_size = 5  # Process chunks in smaller batches
    
    for i in range(0, len(chunks), batch_size):
        print(f"Processing batch {i//batch_size + 1} of {(len(chunks)-1)//batch_size + 1}")
        batch = chunks[i:i+batch_size]
        
        for chunk in batch:
            ball_by_ball_data = chunk.get('ball_by_ball', [])
            
            # Process a limited number of comments per chunk
            max_comments = 10
            for comment in chunk.get("comments", [])[:max_comments]:
                comment_text = comment.get("comment", "")
                if comment_text:
                    sentiment = analyze_sentiment(comment_text)
                    sentiments.append({'comment': comment_text, 'sentiment': sentiment})
            
            betting_opportunity = analyze_betting_opportunity(chunk, ball_by_ball_data)
            betting_opportunities.append({'chunk': chunk, 'betting_opportunity': betting_opportunity})
        
        # Clear memory after each batch
        gc.collect()
        torch.mps.empty_cache() if hasattr(torch.mps, 'empty_cache') else None
    
    # Convert results to DataFrames
    sentiments_df = pd.DataFrame(sentiments)
    betting_opportunities_df = pd.DataFrame(betting_opportunities)
    
    return sentiments_df, betting_opportunities_df

In [10]:
# Process all chunks
print("Starting the processing of chunks...")
sentiments_df, betting_opportunities_df = process_chunks(chunks)
print("Finished processing all chunks.")

# Sentiment distribution
print("\nSentiment Distribution:")
print(sentiments_df['sentiment'].value_counts())

# Sentiment percentages
sentiment_percentages = (
    sentiments_df['sentiment'].value_counts(normalize=True) * 100
).round(2)
print("\n\n")
print("Sentiment Distribution (%):")
print(sentiment_percentages)
print("\n\n")

# Betting opportunities distribution
print("\n\n")
print("Betting Opportunities:")
print(betting_opportunities_df['betting_opportunity'])
print("\n\n")

Starting the processing of chunks...
Processing batch 1 of 11


/Users/darshan/Documents/GitHub/ipl-sentiment-betting/.venv/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/Users/darshan/Documents/GitHub/ipl-sentiment-betting/.venv/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:595: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.8` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/Users/darshan/Documents/GitHub/ipl-sentiment-betting/.venv/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:612: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `20` -- this flag is only used in sample-based generation modes. You should set `do_sa

Analyzing betting opportunity with ball-by-ball data.
Betting opportunity result: given the limited information provided, it seems like we're missing some crucial details such as the specific match, the current standings, and the exact time of the comment. however, based on the available data, i can provide an analysis focusing on the teams' recent performances and player dynamics.

### teams overview:
- **punjab kings (pk)**: the team has been performing well recently, especially with the return of rishabh pant. they have a strong batting lineup, including surya dhawan, jm bairstow, and harpreet brar.
- **delhi capitals (dc)**: they have also had
Analyzing betting opportunity with ball-by-ball data.
Betting opportunity result: given the limited information provided, let's analyze the situation based on the available data:

### team analysis

**punjab kings (home team):**
- **batting lineup:** s dhawan, jm bairstow, ls livingstone, jm sharma†, sm curran, hv patel, harpreet brar, shasha